In [ ]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
data_dir = Path("data/")
output_dir = Path("outputs/")
age_md_path = data_dir / "agedB6_mice_meta.csv"
fi_md_path = data_dir / "df_video_features.csv"
video_list_path = data_dir / "B6J_good_videos_batch_file.txt"

In [ ]:
age_md = pd.read_csv(age_md_path)
print(age_md.columns, age_md.shape)

In [ ]:
fi_md = pd.read_csv(fi_md_path)
print(fi_md.columns, fi_md.shape)

In [ ]:
age_md["DOB_dt"] = pd.to_datetime(age_md["DOB"], format="%m/%d/%y")
age_md["TestDate_dt"] = pd.to_datetime(age_md["TestDate"], format="%m/%d/%y")
age_md["Age_days"] = age_md.apply(lambda row: (row["TestDate_dt"] - row["DOB_dt"]).days, axis=1)
age_md["Age_weeks"] = age_md["Age_days"] // 7
age_md["Age_months"] = age_md["Age_days"] // 30

In [ ]:
### find entries in both tables

merged_md = pd.merge(age_md, fi_md, how="inner", left_on="MouseID", right_on="MouseID")
mask = abs(merged_md["Age_weeks"] - merged_md["TestAge"]) <= 1
md = merged_md[mask].copy()
print(md.columns, md.shape)

In [ ]:
md["path_head"] = md.apply(lambda row: Path(row['NetworkFilename']).with_suffix(""), axis=1)
md["video_path"] = md.apply(lambda row: f"{row['path_head']}_trimmed_overlay.mp4", axis=1)
md["pose_path"] = md.apply(lambda row: f"{row['path_head']}__trimmed_overlay_pose_est_v6.h5", axis=1)

In [ ]:
print(md["video_path"])

In [ ]:
### find entries with valid filepaths

with video_list_path.open("r") as f:
    videos = list(map(lambda x: x.strip()[58:], f.readlines()))
print(videos[:5])
md = md[md["video_path"].isin(videos)].copy()
print(md.columns, md.shape)

In [ ]:
# age_counts = md["Age_months"].value_counts().sort_index()
# age_max = max(md["Age_months"])

# plt.figure()
# plt.bar(age_counts.index, age_counts.values)
# plt.xlabel("Age (months)")
# plt.xticks(np.arange(0, age_max + 1, 6))
# plt.ylabel("Number of Mice")
# plt.show()

# plt.figure()
# plt.bar(age_counts.index, age_counts.values / sum(age_counts.values))
# plt.xlabel("Age (months)")
# plt.xticks(np.arange(0, age_max + 1, 6))
# plt.ylabel("Frequency")
# plt.show()

In [ ]:
# fi_counts = md["score"].value_counts().sort_index()

# plt.figure()
# plt.bar(fi_counts.index, fi_counts.values)
# plt.xlabel("Biological Age (Frailty)")
# plt.ylabel("Number of Mice")
# plt.show()

In [ ]:
md["bucket"] = md.apply(lambda row: min(4, row["Age_months"] // 6) + (5 if row["Sex_y"] == 1 else 0), axis=1)

bucket_counts = md["bucket"].value_counts().sort_index()

plt.figure()
plt.bar(bucket_counts.index, bucket_counts.values)
plt.xlabel("Bucket")
plt.ylabel("Number of Mice")
plt.show()

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, train_size=250, random_state=623)
train_idx, _ = next(sss.split(md, md['bucket']))

stratified_sample = md.iloc[train_idx].reset_index(drop=True)
stratified_sample

In [ ]:
with open(output_dir / "good_video_paths.txt", "w") as f:
    f.writelines("\n".join(list(stratified_sample["video_path"])))
with open(output_dir / "good_pose_paths.txt", "w") as f:
    f.writelines("\n".join(list(stratified_sample["pose_path"])))